# Assignment 25/03/2025

## Imports

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

## Sparse Optical Flow - Lucas-Kanade algorithm

In [ ]:
# REFERENCING: https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html,
#              https://www.geeksforgeeks.org/python-opencv-optical-flow-with-lucas-kanade-method/
video_path = '../videos/cars.mp4'
cap = cv2.VideoCapture(video_path)

In [3]:
# ShiTomasi corner detection
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

In [4]:
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [5]:
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

In [6]:
# Take first frame and find corners in it
ret, old_frame = cap.read()
if not ret:
    print("Error reading video")
    cap.release()
    exit()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

In [7]:
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

In [8]:
# VideoWriter to save output video
height, width, _ = old_frame.shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('lucas_kanade.avi', fourcc, 20.0, (width, height))

In [9]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("No frames grabbed")
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow using Lucas-Kanade method
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
    else:
        print("No points found!")
        break

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    
    img = cv2.add(frame, mask)
    
    # Write the resulting frame into the file
    out.write(img)
    
    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)


# Release all resources
cap.release()
out.release()
print("Video saved")

No frames grabbed
Video saved


## Dense Optical Flow - Farneback's algorithm

In [ ]:
# Set video path and open video capture
video_path = '../videos/globe-earth-animation-4.gif'
cap = cv2.VideoCapture(video_path)

In [11]:
ret, frame1 = cap.read()
if not ret:
    print("Error reading video")
    cap.release()
    exit()

In [12]:
# Prepare initial data for optical flow
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

In [13]:
# Set up VideoWriter to save the optical flow output
height, width, _ = frame1.shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('optical_flow.avi', fourcc, 20.0, (width, height))

In [14]:
while cap.isOpened():
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed')
        break

    next_frame = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate dense optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prvs, next_frame, None, 
                                       0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    out.write(bgr)
    prvs = next_frame.copy()

# Release resources
cap.release()
out.release()
print("Video saved")

No frames grabbed
Video saved
